In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

Here are the basic description of the model, basic parameters of the coffient and the logic of trading.

The action space is {0,1,2,3,4,5,6,7,8,9,10} corresponding to the position of holding {0,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01} btc.

The input of model are 3 fold:market information, current position(previous action) P_p and avaliable trading options.

The output of mdoel are 11 classifications of action.

We choose the classification with the highest value and the corresponding position should be made by taking order P_c.

The order we take is market order and volume should be o_s=P_c-P_p (range from -0.01 to 0.01)

If o_s>0, then we buy o_s btc,if o_s=0, we hold and do not place any order and if o_s<0, we sell -o_s btc.

In [ ]:
action_dim=11
max_punish=1e12

The network of our model

In [ ]:
class masked_net1(nn.Module):
    def __init__(self, N_STATES, N_ACTIONS, hidden_nodes):
        super(masked_net1, self).__init__()
        self.fc1 = nn.Linear(N_STATES, hidden_nodes)
        self.fc2 = nn.Linear(2 * hidden_nodes, hidden_nodes)
        self.out = nn.Linear(hidden_nodes, N_ACTIONS)
        self.embedding = nn.Embedding(N_ACTIONS, hidden_nodes)
        self.register_buffer("max_punish", torch.tensor(max_punish))
        self.init_weights()

    def forward(
        self,
        state: torch.tensor,
        previous_action: torch.tensor,
        avaliable_action: torch.tensor,
    ):
        state_hidden = F.relu(self.fc1(state))
        previous_action_hidden = self.embedding(previous_action)
        information_hidden = torch.cat([state_hidden, previous_action_hidden],
                                       dim=1)
        information_hidden = self.fc2(information_hidden)
        action = self.out(information_hidden)
        masked_action = action + (avaliable_action - 1) * self.max_punish
        return masked_action

    def init_weights(self):
        for _, module in self.named_modules():
            if isinstance(module, nn.Linear):
                torch.nn.init.kaiming_uniform_(module.weight)
                module.bias.data.zero_()


net = masked_net1(66, 11, 128)
net.load_state_dict(torch.load("trained_model.pkl"))

input of the model

In [ ]:
market_state = torch.randn(1, 66)
previous_action = (torch.distributions.Binomial(10, torch.tensor(
        [0.5] * 1)).sample().long())
avaliable_action = torch.bernoulli(torch.Tensor(1, 11).uniform_(0, 1))

NameError: name 'torch' is not defined

In [ ]:
print(market_state.type())
print(market_state.shape)

In [ ]:
print(previous_action.type())
print(previous_action.shape)

In [ ]:
print(avaliable_action.type() )
print(avaliable_action.shape)

In [ ]:
output_action = net(market_state, previous_action, avaliable_action).argmax(dim=1,
                                                             keepdim=True)
output_action 

In [ ]:
os=((output_action-previous_action)*0.01).squeeze().numpy()
os# the os mention before